<a href="https://colab.research.google.com/github/binglesley/Project-90106-G28/blob/master/Research_models(xinhui).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research Data-G28


In [100]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import cross_val_score
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier

This part is loding data set. You just need to modify the filename for different data file. The latest data set is 'cleaned_6_Sep.csv'

In [65]:
def load_data(filename):
    filepath = os.path.join(os.getcwd(), filename)
    data = pd.read_csv(filepath, index_col='Id')
    data['Is_External__c'] = data['Is_External__c'].fillna('Internal')
    try:
      data = data.drop('Status_Reason__c',axis = 1)
    except Exception as e:
      print("No this features")
    #d = data[data['StageName'].isin(['Closed Lost','Closed Won'])]

    return data

This part is help you to delete features which is missing more than 40% and also factorize the catogorical data. It will renturn the training set, labels and also the labels' values.

In [3]:
def Deleting(data):
    for index,col in data.iteritems(): 
        null_proportion = cal_null(col)

    if null_proportion >= 0.4: # if missing more than 40% then remove this feature
        data = data.drop(index, axis=1)
        
    data = data.dropna()

    for index, col in data.iteritems():# finding if it is numerical fetures
        types = type_(col)
        if types == 'catogerical':
            if index == 'StageName':
                labels_corres = pd.factorize(data[index])[1]
                data[index] = pd.factorize(data[index])[0].astype(np.uint16)
            else:
                data[index] = pd.factorize(data[index])[0].astype(np.uint16)

    x = data
    Labels = x['StageName']
    Trains = x.drop('StageName',axis=1)
    #Trains = Trains.drop('Status_Reason__c',axis = 1)
    names = Trains.columns


    return Trains,Labels,names # 'neams' is the list of features' name 


def cal_null(col):
    is_null = col[col.isnull()]
    percentage = len(is_null)/len(col)
    return percentage

def type_(col):
    not_null = col[col.notnull()]
    element = str(not_null[0])
    if element.isdigit():
        return 'number'
    else:
        return 'catogerical'

This function is designed for deal with the original data set and return certain form of data set which could use to imputation.

In [67]:
def deal_data(raw_data):
    missing_set = {}
    no_missing = {}
    index_missing = {}
    index_no_missing = {}
    labels = {}
    missing_names = []
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    

    for index, col in raw_data.iteritems():
      if raw_data[index].isnull().any():
        if len(col[col.isnull()])/len(col) >= 0.4:  # if more than 40% missing, would use the most frequenct records to impute.
            raw_data[index] = imp.fit_transform(raw_data[index].values.reshape(-1, 1))
        else:
            index_missing[index] = col[col.isnull()].index # each missing features' index
            index_no_missing[index] = col[col.notnull()].index
            missing_names.append(index)

    for index,col in raw_data.iteritems():
        if str(col[col.notnull()][0]).isdigit() == False:
            if index == 'StageName':
                labels_corres = pd.factorize(raw_data[index])[1]
            raw_data[index] = pd.factorize(raw_data[index])[0].astype(np.uint16)

    for i in missing_names:
        labels[i] = raw_data.loc[index_no_missing[i],i]


    raw_data = raw_data.dropna(axis = 1)
    

    for key,value in index_missing.items():
        missing_set[key] = raw_data.loc[value]
        no_values = index_no_missing[key]
        no_missing[key] = raw_data.loc[no_values]

    return no_missing,missing_set,index_no_missing,index_missing,labels,missing_names 
    # 'missing_names' is the list of fratures' name which contain missing values 


This part is imputing with random-forest method.

In [81]:
def impute(no_missing,missing_set,index_missing,labels,names,raw_data):
    predict = {}
    features_name = []

    for feature in names:
        label = labels[feature]
        train_set = no_missing[feature]
        missing = missing_set[feature]
        predict[feature] = random_forest(train_set,label,missing)

    for key,items in predict.items():
        c = 0
        for i in index_missing[key]:
            raw_data.loc[i,key] = items[c]
            c += 1
    train = raw_data.drop('StageName',axis=1)
    features_name = [index for index,col in train.iteritems()] #'feature_name' is the list of features' name
    y_label = raw_data['StageName']

    return train,y_label,features_name


def random_forest(train,label,missing):
  random_f = RandomForestRegressor()
  random_f.fit(train,label)
  y = random_f.predict(missing)
  return y

def importances(train,label):
  random_f = RandomForestRegressor()
  random_f.fit(train,label)
  importance = (sorted(zip(map(lambda x: round(x, 4), random_f.feature_importances_),features_name),
             reverse=True)) # the importance ranking of features 
    
  return importance

This part catains two training model, SVM and Decision tree. 

In [83]:
def svc(train_set,label_set,test_set,ground_truth):
    svm_clf = LinearSVC(max_iter=10000)
    svm_clf.fit(train_set, label_set)
    y = svm_clf.predict(test_set)
    p = precision_score(y,ground_truth,average= None)
    r = recall_score(y,ground_truth,average=None)
    f = f1_score(y,ground_truth,average=None)
    print('svm_precision for each labels:',p,'\n')
    print('svm_recall for each labels:',r,'\n')
    print('svm_f1 for each labels:',f,'\n')

    # precision score, recall, F1 score

def decision_tree(train_set,label_set,test_set,ground_truth):
    tree_clf = DecisionTreeClassifier()
    tree_clf.fit(train_set,label_set)
    y = tree_clf.predict(test_set)
    p = precision_score(y, ground_truth, average=None)
    r = recall_score(y, ground_truth, average=None)
    f = f1_score(y, ground_truth, average=None)
    print('tree_precision for each labels:',p,'\n')
    print('tree_recall for each labels:',r,'\n')
    print('tree_f1 for each labels:',f,'\n')

This part build an NN model.

In [98]:
def nn_build(shapes):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_dim=shapes))
    model.add(tf.keras.layers.Dense(300, activation="relu"))
    model.add(tf.keras.layers.Dense(100, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    return model

This part is cross-validation for models.

In [101]:
def k_folds(X_train,y_train,models):
  scores  = cross_val_score(estimator = models,X = X_train,
                      y = y_train,cv = 10,n_jobs = 1)
    print('CV accuracy scores:\n\n %s \n' % scores)
    print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

To check the accuracy of no cross-validation.

In [99]:
raw_data = load_data('cleaned_6_Sep.csv')
no_missing, missing_set, index_no_missing, index_missing, labels, missing_names = deal_data(raw_data)
X_set,y,features_name = impute(no_missing, missing_set, index_missing, labels, missing_names, raw_data)
shapes = X_set.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X_set, y, train_size=0.8, random_state=1)
print('Features importance:',importances(X_train,y_train))
svc(X_train,y_train,X_test,y_test)
decision_tree(X_train,y_train,X_test,y_test)
X_train1,X_dev,y_train1,y_dev = train_test_split(X_train,y_train,train_size=0.8,random_state=1)
model = nn_build(shapes)
model.compile(optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy'])
model.summary()
model.fit(X_train1, y_train1, epochs=10, verbose=True, validation_data=(X_dev, y_dev), batch_size=10)
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("NN Testing Accuracy:  {:.4f}".format(accuracy))


Features importance: [(0.2524, 'CreatedDate'), (0.1132, 'Lead_Faculty__c'), (0.0997, 'CloseDate'), (0.0983, 'AccountId'), (0.0676, 'CloseMonth'), (0.0627, 'CreatedMonth'), (0.0511, 'Lead_School__c'), (0.0442, 'Business_Type__c'), (0.042, 'RecordType.Name'), (0.0344, 'Industry'), (0.0299, 'Industry_Sub_Type__c'), (0.0251, ' Time_length'), (0.0235, 'CloseYear'), (0.0194, 'CreatedYear'), (0.0188, 'RICE_Supported__c'), (0.0093, 'ParentId'), (0.0083, 'Parent_Opportunity__c'), (0.0001, 'RecordType.Name.1'), (0.0, 'Is_External__c'), (0.0, 'Actual_Close_Date__c')]


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


svm_precision for each labels: [0.55667145 0.82800983] 

svm_recall for each labels: [0.84716157 0.52167183] 

svm_f1 for each labels: [0.67186147 0.64007597] 

tree_precision for each labels: [0.80774749 0.67076167] 

tree_recall for each labels: [0.80774749 0.67076167] 

tree_f1 for each labels: [0.80774749 0.67076167] 

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               6300      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 36,501
Trainable params: 36,501
Non-trainable

To check accuracy of cross-validation.

In [102]:
# For SVM model
svm = svm_clf = LinearSVC(max_iter=10000)
k_folds(X_set,y,svm)
# For decision tree
trees = DecisionTreeClassifier()
k_folds(X_set,y,trees)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

CV accuracy scores:

 [0.67753623 0.90217391 0.88949275 0.61413043 0.61413043 0.61231884
 0.38475499 0.66969147 0.64791289 0.70961887] 

CV accuracy: 0.672 +/- 0.140
CV accuracy scores:

 [0.60326087 0.45471014 0.03804348 0.04710145 0.55615942 0.46557971
 0.00181488 0.00181488 0.07259528 0.38656987] 

CV accuracy: 0.263 +/- 0.238
